## Build ONNX models for testing ONNX Runtime
Many of these models are used to test ONNX Runtime's graph transformations.

In [ ]:
import onnx
from onnx import helper, shape_inference
from onnx import AttributeProto, TensorProto, GraphProto
import os.path
import numpy as np

In [ ]:
import pkg_resources
pkg_resources.get_distribution("onnx").version

In [ ]:
def save_model(onnx_graph, name) :
    model_def = helper.make_model(onnx_graph, producer_name='ort-transformations')
    model_def = shape_inference.infer_shapes(model_def)
    onnx.checker.check_model(model_def)
    print('Shape is inferred and model is checked.')
    onnx.save(model_def, os.path.join('gen-models', str(name) + '.onnx'))
    print('Model is saved.')

### Two-transposes

In [ ]:
model_name = 'two-transposes'

# Preprocessing: create a model with two transposes
node1 = helper.make_node('Transpose', ['X'], ['Y'], perm=[1, 0, 2])
node2 = helper.make_node('Transpose', ['Y'], ['Z'], perm=[1, 0, 2])

graph = helper.make_graph(
    [node1, node2],
    model_name,
    [helper.make_tensor_value_info('X', TensorProto.FLOAT, (2, 3, 4))],
    [helper.make_tensor_value_info('Z', TensorProto.FLOAT, (2, 3, 4))],
)

save_model(graph, model_name)

### Abs single node with initializer

In [ ]:
model_name = 'abs-model'

X = np.int8([-127,-4,0,3,127])

# Create one output (ValueInfoProto)
Y = helper.make_tensor_value_info('Y', TensorProto.INT8, X.shape)
X_INFO = helper.make_tensor_value_info('X', TensorProto.INT8, X.shape)

tensor_x = onnx.helper.make_tensor(
                name='X',
                data_type=onnx.TensorProto.INT8,
                dims=X.shape,
                vals=X.tobytes(),raw=True);
# Create a node (NodeProto)
node_def = helper.make_node(
    'Abs',
    inputs=['X'],
    outputs=['Y'],
)

# Create the graph (GraphProto)
graph_def = helper.make_graph(
    [node_def],
    model_name,
    inputs=[],
    outputs=[Y],
    initializer=[tensor_x]
)

save_model(graph_def, model_name)

### Abs and identity model

In [ ]:
model_name = 'abs-id'

node1 = helper.make_node('Abs', ['X'], ['Y'])
node2 = helper.make_node('Identity', ['Y'], ['Z'])

graph = helper.make_graph(
    [node1, node2],
    model_name,
    [helper.make_tensor_value_info('X', TensorProto.FLOAT, (2, 3, 4))],
    [helper.make_tensor_value_info('Z', TensorProto.FLOAT, (2, 3, 4))],
)

save_model(graph, model_name)

### Abs-id-abs model

In [ ]:
model_name = 'abs-id-max'

gnodes = []
gnodes.append(helper.make_node('Abs', ['A'], ['B']))
gnodes.append(helper.make_node('Identity', ['B'], ['C']))
gnodes.append(helper.make_node('Max', ['C'], ['D']))

graph = helper.make_graph(
    gnodes,
    model_name,
    [helper.make_tensor_value_info('A', TensorProto.FLOAT, (2, 3, 4))],
    [helper.make_tensor_value_info('D', TensorProto.FLOAT, (2, 3, 4))],
)

save_model(graph, model_name)

### Shape-abs-id-max

In [ ]:
model_name = 'shape-add'

gnodes = []
gnodes.append(helper.make_node('Shape', ['A'], ['D']))
gnodes.append(helper.make_node('Shape', ['B'], ['E']))
gnodes.append(helper.make_node('Shape', ['C'], ['G']))
gnodes.append(helper.make_node('Add', ['D', 'E'], ['F']))
gnodes.append(helper.make_node('Add', ['F', 'G'], ['H']))
gnodes.append(helper.make_node('Shape', ['H'], ['I']))
gnodes.append(helper.make_node('Identity', ['I'], ['J']))

graph = helper.make_graph(
    gnodes,
    model_name,
    [helper.make_tensor_value_info('A', TensorProto.FLOAT, (2, 3, 4)), 
     helper.make_tensor_value_info('B', TensorProto.FLOAT, (2, 'N', 4)),
     helper.make_tensor_value_info('C', TensorProto.FLOAT, (2, 3, -1))],
    [helper.make_tensor_value_info('J', TensorProto.INT64, (1,))],
)

save_model(graph, model_name)

### Abs-id-id-max model

In [ ]:
model_name = 'abs-2id-max'

gnodes = []
gnodes.append(helper.make_node('Abs', ['A'], ['B']))
gnodes.append(helper.make_node('Identity', ['B'], ['C']))
gnodes.append(helper.make_node('Identity', ['C'], ['D']))
gnodes.append(helper.make_node('Max', ['D'], ['E']))

graph = helper.make_graph(
    gnodes,
    model_name,
    [helper.make_tensor_value_info('A', TensorProto.FLOAT, (2, 3, 4))],
    [helper.make_tensor_value_info('E', TensorProto.FLOAT, (2, 3, 4))],
)

save_model(graph, model_name)

### Slice-elimination model (opset 1)
(**Note**: It will not work with opset 10 or higher.)

In [ ]:
model_name = 'slice-elim'

gnodes = []
gnodes.append(helper.make_node('Abs', ['A'], ['B']))
gnodes.append(helper.make_node('Slice', ['B'], ['C'], axes=[0,1], starts=[0,0], ends=[9223372036854775807,9223372036854775807]))
gnodes.append(helper.make_node('Slice', ['C'], ['D'], starts=[0,0,0], ends=[9223372036854775807,7,9223372036854775807]))
gnodes.append(helper.make_node('Min', ['D'], ['E']))
gnodes.append(helper.make_node('Slice', ['E'], ['F'], axes=[1], starts=[1], ends=[9223372036854775807]))
gnodes.append(helper.make_node('Slice', ['F'], ['G'], axes=[1], starts=[0], ends=[9223372036854775807]))
gnodes.append(helper.make_node('Max', ['G'], ['H']))
gnodes.append(helper.make_node('Slice', ['H'], ['I'], axes=[2], starts=[0], ends=[9223372036854775807]))

graph = helper.make_graph(
    gnodes,
    model_name,
    [helper.make_tensor_value_info('A', TensorProto.FLOAT, (5, 3, 7))],
    [helper.make_tensor_value_info('I', TensorProto.FLOAT, (5, 2, 7))],
)

save_model(graph, model_name)

### Slice-elimination model (opset 10)

In [ ]:
model_name = 'slice-v11-elim'


graph = helper.make_graph(
    [ # nodes
        helper.make_node('Abs', ['A'], ['B']),
        # Will not be removed because of negative starts value.
        helper.make_node('Slice', ['B', 'startsB', 'endsB', 'axesB'], ['C']),
        # Will not be removed because of non-MAX_INT ends value.
        helper.make_node('Slice', ['C', 'startsC', 'endsC'], ['D']),
        helper.make_node('Min', ['D'], ['E']),
        # Will not be removed because of non-1 steps value.
        helper.make_node('Slice', ['E', 'startsE', 'endsE', 'axesE', 'stepsE'], ['F']),
        # Will be removed.
        helper.make_node('Slice', ['F', 'startsF', 'endsF', 'axesF'], ['G']),
        # Will not be removed because of endsG appearing in graph inputs (can be overriden).
        helper.make_node('Slice', ['G', 'startsG', 'endsG'], ['H']),
        helper.make_node('Max', ['H'], ['I']),
        # Will not be removed because node output participates in graph output.
        helper.make_node('Slice', ['I', 'startsI', 'endsI', 'axesI', 'stepsI'], ['J'])
    ],
    "Slice11Elim",  #name
    [  # inputs
        helper.make_tensor_value_info('A', TensorProto.FLOAT, (5, 3, 7)),
        helper.make_tensor_value_info('endsG', TensorProto.INT64, (1,))
    ],
    [  # outputs
        helper.make_tensor_value_info('J', TensorProto.FLOAT, None)
    ],
    [  # initializers
        helper.make_tensor('axesB', TensorProto.INT64, [2], [0,1]),
        helper.make_tensor('startsB', TensorProto.INT64, [2], [0,-1]),
        helper.make_tensor('endsB', TensorProto.INT64, [2], [9223372036854775807,9223372036854775807]),
        helper.make_tensor('startsC', TensorProto.INT64, [3], [0,0,0]),
        helper.make_tensor('endsC', TensorProto.INT64, [3], [9223372036854775807,7,9223372036854775807]),
        helper.make_tensor('axesE', TensorProto.INT64, [2], [0,1]),
        helper.make_tensor('stepsE', TensorProto.INT64, [2], [1,3]),
        helper.make_tensor('startsE', TensorProto.INT64, [2], [0,0]),
        helper.make_tensor('endsE', TensorProto.INT64, [2], [9223372036854775807,9223372036854775807]),
        helper.make_tensor('axesF', TensorProto.INT64, [2], [1,0]),
        helper.make_tensor('startsF', TensorProto.INT64, [2], [0,0]),
        helper.make_tensor('endsF', TensorProto.INT64, [2], [9223372036854775807, 9223372036854775807]),
        helper.make_tensor('startsG', TensorProto.INT64, [1], [0]),
        helper.make_tensor('endsG', TensorProto.INT64, [1], [9223372036854775807]),
        helper.make_tensor('axesI', TensorProto.INT64, [1], [2]),
        helper.make_tensor('stepsI', TensorProto.INT64, [1], [1]),
        helper.make_tensor('startsI', TensorProto.INT64, [1], [0]),
        helper.make_tensor('endsI', TensorProto.INT64, [1], [9223372036854775807])
    ]
)

save_model(graph, model_name)

### Zero-node graph

In [ ]:
model_name = 'zero-node'

gnodes = []

X = np.int8([-127,-4,0,3,127])

graph = helper.make_graph(
    [],
    model_name,
    [helper.make_tensor_value_info('A', TensorProto.INT8, X.shape)],
    [helper.make_tensor_value_info('A', TensorProto.INT8, X.shape)],
    []
)

save_model(graph, model_name)

### Simple add graph with batch dimension

In [ ]:
model_name = 'simple-add-sym'


node1 = helper.make_node('Add', ['X', 'Y'], ['Z'])

graph = helper.make_graph(
    [node1],
    model_name,
    [helper.make_tensor_value_info('X', TensorProto.INT32, ("sym",1)), helper.make_tensor_value_info('Y', TensorProto.INT32, ("sym",1))],
    [helper.make_tensor_value_info('Z', TensorProto.INT32, ("sym", 1))],
)

save_model(graph, model_name)